<a href="https://colab.research.google.com/github/prisha90/Study-Oriented-Project/blob/main/Glaucoma_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage import morphology

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
IMAGES_FOLDER = "/content/drive/MyDrive/SOP (Rajya Laksmi Ma'am)/Work Document/G1020/Images"
df = pd.read_csv("/content/drive/MyDrive/SOP (Rajya Laksmi Ma'am)/Work Document/G1020/G1020.csv")

images = []
labels = []
image_size = (512, 512)

print(f"Total images in dataframe: {len(df)}")

if not os.path.exists(IMAGES_FOLDER):
    print(f"Error: IMAGES_FOLDER not found: {IMAGES_FOLDER}")
else:
    print(f"IMAGES_FOLDER found: {IMAGES_FOLDER}")

missing_images = [img for img in df['imageID'] if not os.path.exists(os.path.join(IMAGES_FOLDER, img))]
if missing_images:
    print(f"Missing images: {len(missing_images)} images not found.")

def rotate_image(image, angle):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, rot_matrix, (w, h))
    rotated = rotated.reshape(rotated.shape[0], rotated.shape[1], 1)
    return rotated

Total images in dataframe: 1020
IMAGES_FOLDER found: /content/drive/MyDrive/SOP (Rajya Laksmi Ma'am)/Work Document/G1020/Images


In [ ]:
for index, row in df.iterrows():
    img_name = row['imageID']
    label = row['binaryLabels']
    img_path = os.path.join(IMAGES_FOLDER, img_name)

    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img, (65, 65), 0)
        img = img.reshape(image_size[0], image_size[1], 1)

        images.append(img)
        labels.append(label)

        for angle in [45, 90, 170]:
           rotated_img = rotate_image(img, angle)
           images.append(rotated_img)
           labels.append(label)
    else:
        print(f"Image not found: {img_path}")

images = np.array(images) / 255.0
labels = np.array(labels)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
def build_unet():
    inputs = layers.Input((512, 512, 1))

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    p1 = layers.Dropout(0.5)(p1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    p2 = layers.Dropout(0.5)(p2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    p3 = layers.Dropout(0.5)(p3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = layers.BatchNormalization()(c4)

    u1 = layers.UpSampling2D((2, 2))(c4)
    u1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.BatchNormalization()(u1)
    d1 = layers.Concatenate()([u1, c3])

    u2 = layers.UpSampling2D((2, 2))(d1)
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.BatchNormalization()(u2)
    d2 = layers.Concatenate()([u2, c2])

    u3 = layers.UpSampling2D((2, 2))(d2)
    u3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    u3 = layers.BatchNormalization()(u3)
    d3 = layers.Concatenate()([u3, c1])

    od_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='optic_disc')(d3)
    oc_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='optic_cup')(d3)

    return models.Model(inputs, [od_output, oc_output])

    unet = build_unet()

    unet.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0004, momentum=0.95),
             loss='binary_crossentropy',
             metrics=['accuracy'])



In [ ]:
od_train_dummy = np.zeros((len(X_train), 512, 512, 1))
oc_train_dummy = np.zeros((len(X_train), 512, 512, 1))
od_val_dummy = np.zeros((len(X_val), 512, 512, 1))
oc_val_dummy = np.zeros((len(X_val), 512, 512, 1))

unet.fit(X_train, [od_train_dummy, oc_train_dummy],
         epochs=150, batch_size=2,
         validation_data=(X_val, [od_val_dummy, oc_val_dummy]))

od_pred, oc_pred = unet.predict(X_test)

In [ ]:
def calculate_cdr(disc_mask, cup_mask):
    disc_area = np.sum(disc_mask)
    cup_area = np.sum(cup_mask)

    return round((2 * cup_area) / disc_area, 4) if disc_area != 0 else 0

def extract_isnt_quadrants(disc_mask, cup_mask):
    disc_mask = disc_mask.squeeze()
    cup_mask = cup_mask.squeeze()

    disc_mask_rotated = np.rot90(disc_mask)
    cup_mask_rotated = np.rot90(cup_mask)

    nrr_mask = cv2.bitwise_xor(disc_mask_rotated, cup_mask_rotated)

    height, width = disc_mask.shape
    I = np.sum(nrr_mask[height//2:, :])
    S = np.sum(nrr_mask[:height//2, :])
    N = np.sum(nrr_mask[:, :width//2])
    T = np.sum(nrr_mask[:, width//2:])

    return round((1 + (I + S)) / (1 + (N + T)), 4) if (N + T) != 0 else 0

def extract_blood_vessels(fundus_image):
    if len(fundus_image.shape) == 2:
        fundus_image = cv2.cvtColor(fundus_image, cv2.COLOR_GRAY2RGB)

    gray_channel = fundus_image[:, :, 1].astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(6,6))
    enhanced = clahe.apply(gray_channel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    bottom_hat = cv2.morphologyEx(enhanced, cv2.MORPH_BLACKHAT, kernel)

    threshold_value = 3.15 * np.std(bottom_hat)
    _, vessel_mask = cv2.threshold(bottom_hat, threshold_value, 255, cv2.THRESH_BINARY)

    height, width = vessel_mask.shape
    I = np.sum(vessel_mask[height//2:, :])
    S = np.sum(vessel_mask[:height//2, :])
    N = np.sum(vessel_mask[:, :width//2])
    T = np.sum(vessel_mask[:, width//2:])

    return round((1 + (I + S)) / (1 + (N + T)), 4) if (N + T) != 0 else 0

In [ ]:
cdr_test = np.array([calculate_cdr(od, oc) for od, oc in zip(od_pred, oc_pred)])
isnt_test = np.array([extract_isnt_quadrants(od, oc) for od, oc in zip(od_pred, oc_pred)])
bvr_test = np.array([extract_blood_vessels(img) for img in X_test])

X_test_combined = np.hstack((cdr_test.reshape(-1,1), isnt_test.reshape(-1,1), bvr_test.reshape(-1,1)))

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(X_train_combined, y_train)
svm_predictions = svm.predict(X_test_combined)

mlp = MLPClassifier(hidden_layer_sizes=(3, 2), max_iter=120, activation='relu', solver='adam')
mlp.fit(X_train_combined, y_train)
mlp_predictions = mlp.predict(X_test_combined)

adaboost = AdaBoostClassifier(n_estimators=50)
adaboost.fit(X_train_combined, y_train)
adaboost_predictions = adaboost.predict(X_test_combined)


In [ ]:
svm_accuracy = accuracy_score(y_test, svm_predictions)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
adaboost_accuracy = accuracy_score(y_test, adaboost_predictions)
print(f"SVM Accuracy: {svm_accuracy}")
print(f"MLP Accuracy: {mlp_accuracy}")
print(f"AdaBoost Accuracy: {adaboost_accuracy}")